In [70]:
#Import Necessary Libraries
import pandas as pd

In [71]:
# Function to clean column names while loading data
def clean_column_names(df):
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    return df

# Load datasets
consumer_prices = clean_column_names(pd.read_csv('Consumer prices indicators - FAOSTAT_data_en_2-22-2024.csv'))
crops_production = clean_column_names(pd.read_csv('Crops production indicators - FAOSTAT_data_en_2-22-2024.csv'))
emissions = clean_column_names(pd.read_csv('Emissions - FAOSTAT_data_en_2-27-2024.csv'))
employment = clean_column_names(pd.read_csv('Employment - FAOSTAT_data_en_2-27-2024.csv'))
exchange_rate = clean_column_names(pd.read_csv('Exchange rate - FAOSTAT_data_en_2-22-2024.csv'))
fertilizers_use = clean_column_names(pd.read_csv('Fertilizers use - FAOSTAT_data_en_2-27-2024.csv'))
food_balances = clean_column_names(pd.read_csv('Food balances indicators - FAOSTAT_data_en_2-22-2024.csv'))
food_security = clean_column_names(pd.read_csv('Food security indicators  - FAOSTAT_data_en_2-22-2024.csv'))
food_trade = clean_column_names(pd.read_csv('Food trade indicators - FAOSTAT_data_en_2-22-2024.csv'))
foreign_investment = clean_column_names(pd.read_csv('Foreign direct investment - FAOSTAT_data_en_2-27-2024.csv'))
land_temp_change = clean_column_names(pd.read_csv('Land temperature change - FAOSTAT_data_en_2-27-2024.csv'))
land_use = clean_column_names(pd.read_csv('Land use - FAOSTAT_data_en_2-22-2024.csv'))
pesticides_use = clean_column_names(pd.read_csv('Pesticides use - FAOSTAT_data_en_2-27-2024.csv'))

# Dictionary of dataset names and dataframes
datasets = {
    "Consumer Prices": consumer_prices,
    "Crops Production": crops_production,
    "Emissions": emissions,
    "Employment": employment,
    "Exchange Rate": exchange_rate,
    "Fertilizers Use": fertilizers_use,
    "Food Balances": food_balances,
    "Food Security": food_security,
    "Food Trade": food_trade,
    "Foreign Investment": foreign_investment,
    "Land Temperature Change": land_temp_change,
    "Land Use": land_use,
    "Pesticides Use": pesticides_use
}


C:\Users\PMLS\AppData\Local\Temp\ipykernel_2304\1149017488.py:18: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  land_use = clean_column_names(pd.read_csv('Land use - FAOSTAT_data_en_2-22-2024.csv'))


In [72]:

# Inspect the columns in each dataset
for table_name, df in datasets.items():
    print(f"Columns in {table_name} dataframe:")
    print(df.columns, "\n")
# Function to process dataframe
def process_dataframe(df, table_name):
    # Ensure 'year' and 'area' columns are of the same data type
    df['year'] = df['year'].astype(str)
    df['area'] = df['area'].astype(str)
    
    if 'year' in df.columns and 'area' in df.columns and 'value' in df.columns:
        grouped_df = df.groupby(['year', 'area'])['value'].mean().reset_index()
        new_column_name = f"{table_name}_value"
        grouped_df.rename(columns={'value': new_column_name}, inplace=True)
        return grouped_df
    else:
        print(f"Skipping dataframe due to missing 'year', 'area', or 'value' columns.")
        return None
# Process each dataframe
processed_dfs = {}
for table_name, df in datasets.items():
    processed_df = process_dataframe(df, table_name)
    if processed_df is not None:
        processed_dfs[table_name] = processed_df

# Display the first few rows of each processed dataframe
for table_name, df in processed_dfs.items():
    print(f"Processed {table_name} dataframe:")
    print(df.head(), "\n")

Columns in Consumer Prices dataframe:
Index(['domain_code', 'domain', 'area_code_(m49)', 'area', 'year_code', 'year',
       'item_code', 'item', 'months_code', 'months', 'element_code', 'element',
       'unit', 'value', 'flag', 'flag_description', 'note'],
      dtype='object') 

Columns in Crops Production dataframe:
Index(['domain_code', 'domain', 'area_code_(m49)', 'area', 'element_code',
       'element', 'item_code_(cpc)', 'item', 'year_code', 'year', 'unit',
       'value', 'flag', 'flag_description', 'note'],
      dtype='object') 

Columns in Emissions dataframe:
Index(['domain_code', 'domain', 'area_code_(m49)', 'area', 'element_code',
       'element', 'item_code_(cpc)', 'item', 'year_code', 'year',
       'source_code', 'source', 'unit', 'value', 'flag', 'flag_description',
       'note'],
      dtype='object') 

Columns in Employment dataframe:
Index(['domain_code', 'domain', 'area_code_(m49)', 'area', 'indicator_code',
       'indicator', 'sex_code', 'sex', 'year_code', 

In [73]:

# Merge all processed dataframes on 'year' and 'area'
merged_df = None

for table_name, df in processed_dfs.items():
    if merged_df is None:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on=['year', 'area'], how='outer')

# Display the first few rows of the merged dataframe
print("Merged DataFrame:")
merged_df.info()


Merged DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14555 entries, 0 to 14554
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   year                           14555 non-null  object 
 1   area                           14555 non-null  object 
 2   Consumer Prices_value          4856 non-null   float64
 3   Crops Production_value         4587 non-null   float64
 4   Emissions_value                5130 non-null   float64
 5   Employment_value               4217 non-null   float64
 6   Exchange Rate_value            8639 non-null   float64
 7   Fertilizers Use_value          1933 non-null   float64
 8   Food Balances_value            2177 non-null   float64
 9   Food Security_value            8580 non-null   float64
 10  Food Trade_value               6205 non-null   float64
 11  Foreign Investment_value       4580 non-null   float64
 12  Land Temperature Change_valu

In [74]:
# Drop rows where 'Land Use_value' has NaN values
if 'Land Use_value' in merged_df.columns:
    merged_df = merged_df.dropna(subset=['Land Use_value'])

# Count the values in the 'year' column
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9519 entries, 0 to 14554
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   year                           9519 non-null   object 
 1   area                           9519 non-null   object 
 2   Consumer Prices_value          4397 non-null   float64
 3   Crops Production_value         4388 non-null   float64
 4   Emissions_value                5119 non-null   float64
 5   Employment_value               4176 non-null   float64
 6   Exchange Rate_value            7969 non-null   float64
 7   Fertilizers Use_value          1933 non-null   float64
 8   Food Balances_value            2177 non-null   float64
 9   Food Security_value            4434 non-null   float64
 10  Food Trade_value               6009 non-null   float64
 11  Foreign Investment_value       4350 non-null   float64
 12  Land Temperature Change_value  4868 non-null   float

In [75]:
merged_df.head(9519)

,year,area,Consumer Prices_value,Crops Production_value,Emissions_value,Employment_value,Exchange Rate_value,Fertilizers Use_value,Food Balances_value,Food Security_value,Food Trade_value,Foreign Investment_value,Land Temperature Change_value,Land Use_value,Pesticides Use_value
0,2000,Afghanistan,26.629848,60177.909091,3.592117,2765.95,47357.574730,NaN,NaN,30.4200,18485.588235,0.170000,0.9128,28320.888889,NaN
1,2000,Albania,57.765107,85900.272727,18.502933,557.29,143.709417,NaN,NaN,25.8400,12450.391304,144.270315,0.9002,1120.666667,67.618889
2,2000,Algeria,50.779359,46022.000000,0.226350,1392.48,75.259792,NaN,NaN,18.7425,109091.273750,146.900000,0.6867,67244.866667,655.343333
3,2000,Andorra,71.219256,NaN,0.000000,NaN,1.085180,NaN,NaN,6.8900,NaN,NaN,0.8999,23.428571,6.752500
4,2000,Angola,1.691044,65443.181818,58.137783,2487.37,10.040544,NaN,NaN,27.5400,18634.125000,878.601000,0.2226,42910.344444,11.432857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14550,2019,Saint Barthélemy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN
14551,2020,"Bonaire, Sint Eustatius and Saba",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.200000,NaN
14552,2020,Saint Barthélemy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN
14553,2021,"Bonaire, Sint Eustatius and Saba",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.200000,NaN


In [76]:
# Fill NaN values with the mean of each numeric column
numeric_cols = merged_df.select_dtypes(include=['number']).columns
merged_df[numeric_cols] = merged_df[numeric_cols].fillna(merged_df[numeric_cols].mean())

# Display the information of the final merged dataframe
merged_df.info()

# Display the first few rows of the final merged dataframe
print(merged_df.head(190))

<class 'pandas.core.frame.DataFrame'>
Index: 9519 entries, 0 to 14554
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   year                           9519 non-null   object 
 1   area                           9519 non-null   object 
 2   Consumer Prices_value          9519 non-null   float64
 3   Crops Production_value         9519 non-null   float64
 4   Emissions_value                9519 non-null   float64
 5   Employment_value               9519 non-null   float64
 6   Exchange Rate_value            9519 non-null   float64
 7   Fertilizers Use_value          9519 non-null   float64
 8   Food Balances_value            9519 non-null   float64
 9   Food Security_value            9519 non-null   float64
 10  Food Trade_value               9519 non-null   float64
 11  Foreign Investment_value       9519 non-null   float64
 12  Land Temperature Change_value  9519 non-null   float

In [77]:
# Save the merged dataframe to a CSV file
merged_df.to_csv('data.csv', index=False)

Model Deployment

In [98]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Select numerical and categorical columns
numerical_cols = X.select_dtypes(include=np.number).columns
categorical_cols = X.select_dtypes(exclude=np.number).columns

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Apply preprocessing to training data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Define the MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_preprocessed.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer with linear activation for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_preprocessed, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss = model.evaluate(X_test_preprocessed, y_test)
print("Test Loss:", test_loss)

C:\Users\PMLS\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 12356158464.0000 - val_loss: 13008567296.0000
Epoch 2/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 12232576000.0000 - val_loss: 12888993792.0000
Epoch 3/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 11984629760.0000 - val_loss: 12535864320.0000
Epoch 4/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 11880068096.0000 - val_loss: 11878747136.0000
Epoch 5/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 11074952192.0000 - val_loss: 10898290688.0000
Epoch 6/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 10134807552.0000 - val_loss: 9645410304.0000
Epoch 7/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8841261056.0000 - val_loss: 8224036864.0000
Epoch 8/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7397836288.0000 - val_loss: 6756516352.0000
Epoch 9/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 5820659712.0000 - val_loss: 5396606976.0000
Epoch 10/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step 

In [99]:
# Evaluate the model
test_loss = model.evaluate(X_test_preprocessed, y_test)
print("Test Loss (MSE):", test_loss)

# Compute RMSE
rmse = np.sqrt(test_loss)
print("Root Mean Squared Error (RMSE):", rmse)


60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 934255040.0000
Test Loss (MSE): 839949824.0
Root Mean Squared Error (RMSE): 28981.887861214287


In [101]:
import pandas as pd

# Assuming you have true labels for the test instances (replace true_labels with your actual true labels)
true_labels = y_test  # Assuming y_test contains the true labels for test instances

# Use the trained model to make predictions on the test data
predictions = model.predict(X_test_preprocessed)

# Create a DataFrame with instance IDs, true labels, and predicted values
predictions_df = pd.DataFrame({
    'Instance_ID': range(len(predictions)),  # Generate a sequence of numbers as identifiers
    'True_Label': true_labels.values.flatten(),  # Convert the Series to a numpy array and flatten
    'Predicted_Value': predictions.flatten()
})

# Display the prediction DataFrame
print(predictions_df)

# Save the DataFrame to a CSV file
predictions_df.to_csv('predictions.csv', index=False)

60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
      Instance_ID     True_Label  Predicted_Value
0               0   39124.800000     78930.617188
1               1  224000.400000    197988.703125
2               2  104604.453088     84264.148438
3               3  104604.453088    105885.914062
4               4  104604.453088     62857.210938
...           ...            ...              ...
1899         1899  104604.453088    121885.117188
1900         1900   62413.000000     84544.234375
1901         1901  104604.453088     72414.609375
1902         1902   84719.444444     94440.539062
1903         1903  104604.453088    105977.742188

[1904 rows x 3 columns]


In [102]:
total_instances = len(data)
print("Total number of instances:", total_instances)

Total number of instances: 9519


In [103]:
# Number of instances in training and test sets
num_train_instances = len(X_train)
num_test_instances = len(X_test)

print("Number of instances in the training set:", num_train_instances)
print("Number of instances in the test set:", num_test_instances)

Number of instances in the training set: 7615
Number of instances in the test set: 1904


In [105]:
model.compile(optimizer='adam', loss='mean_squared_error')